In [2]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
import random as rd
from sklearn.decomposition import PCA
from IPython.display import display
import os 
from pathlib import Path
import shutil 


# En este notebook conseguimos el shape feature (b_shape) de nuestras imagenes 


## Assumptions :

Cada secuencia muestra 1 de las 6 emociones básicas

La primer frame de cada secuencia muestra  la cara en una expression neutral

La intensidad de la expresion aumenta en cada frame de un sujeto (siendo intesidad 1 el ultimo frame. Otros no asumen la mayor intensidad en ultimo frame, nosotros si)

Aca para imagen nueva vamos a tener que usar aam para conseguir los landmarks. Por ahora vamos a extraer el feature 
b_shape para nuestra c_mock que usaremos de entrenamiento (mock)

we need to understansd the landmarks (annotaste) and the aam for the test image (input images)
#TO DO

We take the landmarks from the images of the ck+ database

In [6]:
subjects_landmarks = '../ck+copy/Landmarks/'
sorted_subjects =   np.sort(os.listdir('../ck+copy/Landmarks/')) # we probably need to not have the sorted

distances = []
for subject in sorted_subjects:
    one_subject_landmarks = '../ck+copy/Landmarks/' + str(subject)+ '/'
    sorted_list = np.sort(os.listdir(one_subject_landmarks)) 
    distances_per_subject = []
    for emotion in sorted_list:
        matrix_landmark_per_frame = []
        subject_landmark_emotion = one_subject_landmarks + str(emotion)    
          

In [ ]:
subjects_landmarks = '../ck+copy/Landmarks/'
sorted_subjects =   np.sort(os.listdir('../ck+copy/Landmarks/')) # we probably need to not have the sorted

distances = []
for subject in sorted_subjects:
    one_subject_landmarks = '../ck+copy/Landmarks/' + str(subject)+ '/'
    sorted_list = np.sort(os.listdir(one_subject_landmarks)) #creo que este sort se puede quitar
    distances_per_subject = []
    for emotion in sorted_list:
        matrix_landmark_per_frame = []
        subject_landmark_emotion = one_subject_landmarks + str(emotion)    
        for i in range(0, len(os.listdir(subject_landmark_emotion))):
            data = pd.read_csv('../ck+copy/Landmarks/' +  str(subject)+ '/' + str(emotion)+'/'+ str(subject)+ '_'+str(emotion)+'_'+ str(i+1).zfill(8)+'_landmarks.txt',  header=None)
            data.columns =['landmarks']
            quantity_of_landmarks = data.size #68
            data = data['landmarks'].str.split('   ', expand=True)
            data.columns =['landmarks', 'landmarkX', 'landmarkY']
            data =data.drop(columns=['landmarks'])
            landmarkXY=data['landmarkX'].append(data['landmarkY'], ignore_index=True)
            landmarkXY_array =landmarkXY.to_numpy(dtype=float)
            matrix_landmark_per_frame.append(landmarkXY_array)
        frame0 = matrix_landmark_per_frame[0]
        distance_landmark_per_frame = matrix_landmark_per_frame
        for i in range(0, len(distance_landmark_per_frame)):
            distance_landmark_per_frame[i] = distance_landmark_per_frame[i] - frame0
        distances_per_subject.append(distance_landmark_per_frame)
    distances.append(distances_per_subject)
#tenemos matrix que por cada sujeto tiene una matriz con cada emocion, que por cada distancia por emocion tiene vector con la distancia de frames con el primero por emocion

In [8]:
distances = []
for subject in sorted_subjects:
    one_subject_landmarks = '../ck+copy/Landmarks/' + str(subject)+ '/'
    sorted_list = np.sort(os.listdir(one_subject_landmarks)) #creo que este sort se puede quitar
    distances_per_subject = []
    for emotion in sorted_list:
        matrix_landmark_per_frame = []
        subject_landmark_emotion = one_subject_landmarks + str(emotion)    
        for i in range(0, len(os.listdir(subject_landmark_emotion))):
            data = pd.read_csv('../ck+copy/Landmarks/' +  str(subject)+ '/' + str(emotion)+'/'+ str(subject)+ '_'+str(emotion)+'_'+ str(i+1).zfill(8)+'_landmarks.txt',  header=None)
            data.columns =['landmarks']
            quantity_of_landmarks = data.size #68
            data = data['landmarks'].str.split('   ', expand=True)
            data.columns =['landmarks', 'landmarkX', 'landmarkY']
            data =data.drop(columns=['landmarks'])
            landmarkXY=data['landmarkX'].append(data['landmarkY'], ignore_index=True)
            landmarkXY_array =landmarkXY.to_numpy(dtype=float)
            matrix_landmark_per_frame.append(landmarkXY_array)        
        distance_landmark_per_frame = matrix_landmark_per_frame       
        distances_per_subject.append(distance_landmark_per_frame)
    distances.append(distances_per_subject)
#tenemos matrix que por cada sujeto tiene una matriz con cada emocion, que por cada distancia por emocion tiene vector con la distancia de frames con el primero por emocion

### Aca convertimos  en array y agrupamos por secuencia de imagenes

In [9]:
for i in range(0, len(distances)):
    for j in range(0, len( distances[i])):
        distances[i][j]= np.asarray( distances[i][j], dtype=object)
    distances[i]= np.asarray( distances[i], dtype=object)
distances= np.asarray( distances, dtype=object)
print(distances[0][0].shape)

(29, 136)


### Aca  convertimos en array y apilamos todos los frames de todas las secuencias

In [10]:
stacked_array = []
for i in range(0, len(distances)):
    stacked_array.append(np.vstack(tuple(distances[i])))
stacked_array= np.asarray( stacked_array, dtype=object)
total_stacked_landmarkRawData = np.vstack(tuple(stacked_array))
print(total_stacked_landmarkRawData.shape)

(965, 136)


In [29]:
my_df = pd.DataFrame(total_stacked_landmarkRawData)
my_df.to_csv('ckmock_landmark_rawData.csv',header = False, index= False)

### Aca  separamos las primeras frames de cada secuencia, ( cara  neutral), y la cantidad de frames por secuencia, para poder luego sacar las diferencia entre los landmarks de los frames de una secuencia  y el primer frame de esta (cara neutral). 

In [24]:
first_frame =[]
cant_frame_per_seq = []
for i in range(0, len(distances)):
    for j in range(0, len( distances[i])):
        first_frame.append(distances[i][j][0])
        cant_frame_per_seq.append(len(distances[i][j]))
        

first_frame = np.asarray( first_frame, dtype=object)
cant_frame_per_seq = np.asarray( cant_frame_per_seq, dtype=object)

In [31]:
first_frame.shape, cant_frame_per_seq


((51, 136),
 array([29, 16, 30, 19, 17, 30, 17, 33, 23, 32, 17, 19, 12, 29, 25, 14, 24,
        15, 14, 10, 21, 13, 29, 28, 14, 21, 14, 33, 17, 19, 27, 22, 20, 13,
        14, 15, 13, 5, 23, 16, 14, 15, 7, 6, 27, 20, 10, 22, 9, 15, 18],
       dtype=object))

In [32]:
my_df = pd.DataFrame(first_frame)
my_df.to_csv('ckmock_landmark_firstFrame.csv',header = False, index= False)

In [33]:
my_df = pd.DataFrame(cant_frame_per_seq)
my_df.to_csv('ckmock_landmark_FramePerSeq.csv',header = False, index= False)

# Alineación de shapes con procustres.

## Ahora, para que todas nuestras imagenes estén alineadas, utilizamos Procustres analysis. 

Este analisis, en su version normal, utiliza una imagen de referencia y luego encuentra las transformaciones de rotacion,  translacion y escala que minimizen la diferencia con esa imagen 

(La versión generalizada en cambio ve las distancias de la aproximacion de entre todos los pares y elije la imagen cuyas distancias son menores ) 

In [ ]:
EL procustres de spicy devuelve los landmarks corridos, pero no devuelve 
las matrices de trasnformación, que son necesarias para luego hacer warp de las imagenes
en tiempo polinomial.

Acá primero lo hacemos con spicy (normal) y luego con un computo de procustres.
(normal y generalizado)

Vemos las diferencias y como dan los b_shapes con cada uno

In [ ]:
we have todo procustres and we need noormalization of the landmark differences, (la normalizacion es la resta)

In [34]:
my_data = pd.read_csv('ckmock_landmark_rawData.csv',header = None)
my_data #estas son las shapes sin la diferencia [x1...xl,y1..yl]

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,245.68800,250.75381,259.36702,266.68465,277.26378,295.88027,319.49635,346.07042,375.99423,406.69956,...,372.84879,368.30900,358.95264,357.22541,356.73931,357.43714,359.24213,360.59360,361.01155,360.49401
1,245.68800,250.75381,259.36702,266.68465,277.26378,295.88027,319.49635,346.07042,375.99423,406.69956,...,372.84879,368.30900,358.95264,357.22541,356.73931,357.43714,359.24213,360.59360,361.01155,360.49401
2,245.68800,250.75381,259.36702,266.68465,277.26378,295.88027,319.49635,346.07042,375.99423,406.69956,...,372.84879,368.30900,358.95264,357.22541,356.73931,357.43714,359.24213,360.59360,361.01155,360.49401
3,245.52430,250.60603,259.24738,266.58873,277.18849,295.83317,319.48002,346.10317,376.06987,406.81441,...,372.94551,368.40449,359.04407,357.29451,356.80126,357.49982,359.32130,360.69453,361.13300,360.60285
4,245.63147,250.71666,259.34747,266.69404,277.31085,295.95780,319.60644,346.21376,376.18077,406.91947,...,372.97670,368.39557,358.98119,357.21598,356.71773,357.41864,359.25529,360.66201,361.11404,360.56986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,193.60994,191.96195,191.42480,194.13260,207.74205,230.90409,258.54349,287.82045,321.88795,358.46063,...,373.03568,366.71602,349.56848,349.36120,351.31250,349.56220,349.68906,349.81593,352.25400,349.77576
961,193.57935,191.91649,191.40544,194.11941,207.71431,230.91723,258.56952,287.86400,321.94880,358.54341,...,373.32072,367.02409,349.84369,349.65369,351.55793,349.88613,349.97891,350.07169,352.45317,350.03369
962,193.41618,191.77914,191.30121,194.05367,207.68432,230.93460,258.64083,287.95782,322.07101,358.69361,...,373.46142,367.19207,350.00780,349.83455,351.72253,350.01896,350.08910,350.15924,352.56142,350.18105
963,193.45437,191.84596,191.33606,194.08412,207.74749,230.95040,258.64575,287.96100,322.06519,358.66876,...,373.36790,367.05972,349.90446,349.69817,351.64340,349.82305,349.95286,350.08266,352.57673,350.11074


### En estas 2 celdas cambiamos el formato, para tener los datos en forma intercalada (x1,y1,x2,y2) y xy (x1,x2..xn,y1,y2..yn). Usamos este último para b_shape como dice el paper, pero para procuestres lo usamos en orden intercalado (de a pares)

In [63]:
#Cambiamos a [x1, y1, x2,y2]
intercalated_data =[]
for i in range(0,len(my_data)):
    x_coord =my_data.iloc[i][0:int(len(my_data.iloc[i])/2)]
    y_coord = my_data.iloc[i][int(len(my_data.iloc[i])/2):len(my_data.iloc[i])]
    output = []
    for (a, b) in zip(x_coord, y_coord):
        output.append(a)
        output.append(b)
    intercalated_data.append(np.asarray(output))
intercalated_data = np.asarray(intercalated_data)
intercalated_data.shape

(965, 136)

In [64]:
my_df = pd.DataFrame(intercalated_data)
my_df.to_csv('ckmock_landmark_intercalated_data.csv',header = False, index= False)

In [65]:
#Ahora lo hacemos en pares
#Cambiamos a [x1, y1, x2,y2]
intercalated_data =[]
for i in range(0,len(my_data)):
    x_coord =my_data.iloc[i][0:int(len(my_data.iloc[i])/2)]
    y_coord = my_data.iloc[i][int(len(my_data.iloc[i])/2):len(my_data.iloc[i])]
    output = []
    for (a, b) in zip(x_coord, y_coord):
        output.append(np.array([a,b]))        
    intercalated_data.append(np.asarray(output))
intercalated_data = np.asarray(intercalated_data)
intercalated_data.shape

(965, 68, 2)

In [ ]:
Procustres
#Usamos la primera imagen como shape de referencia (puede generalizarse)

In [70]:

from scipy.spatial import procrustes

aligned_data = []
for i in range(1, len(intercalated_data)):
    mtx1, mtx2, disparity = procrustes(intercalated_data[0], intercalated_data[i])
    if(i == 1):
        aligned_data.append(mtx1)
        aligned_data.append(mtx2)
    else:
        aligned_data.append(mtx2)


68

In [ ]:
aligned_data[0]

In [75]:
aligned_data[0].flatten().shape

(136,)

In [79]:
aligned_data_intercalated = []
for i in range(0, len(aligned_data)):
    aligned_data_intercalated.append(aligned_data[i].flatten())
#np.asarray(aligned_data_intercalated).shape
aligned_data_intercalated = np.asarray(aligned_data_intercalated)

(965, 136)

In [80]:
my_df = pd.DataFrame(aligned_data_intercalated)
my_df.to_csv('ckmock_landmark_aligned_intercalated_data.csv',header = False, index= False)

In [84]:
aligned_data_xy = []
for i in range(0, len(aligned_data)):
    aligned_data_xy.append(aligned_data[i].flatten('F'))
#np.asarray(aligned_data_intercalated).shape
aligned_data_xy = np.asarray(aligned_data_xy)
aligned_data_xy[0][68]

-0.06800458899401303

In [86]:
my_df = pd.DataFrame(aligned_data_xy)
my_df.to_csv('ckmock_landmark_aligned_xy.csv',header = False, index= False)

### Ahora si, trabajamos con esta data alineada en orden xy. Luego de cargala, conseguimos la diferencia con cada frame 0 por secuencia, y con esos datos utilizamos PCA para  encontrar los b_shape, que tienen dimension #principal_component x 1 para cada frame . 

In [2]:
my_data = pd.read_csv('ckmock_landmark_aligned_xy.csv',header = None)
my_data

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,-0.163450,-0.156895,-0.145752,-0.136284,-0.122597,-0.098510,-0.067956,-0.033574,0.005142,0.044869,...,0.087766,0.081893,0.069788,0.067553,0.066924,0.067827,0.070162,0.071911,0.072451,0.071782
1,-0.163450,-0.156895,-0.145752,-0.136284,-0.122597,-0.098510,-0.067956,-0.033574,0.005142,0.044869,...,0.087766,0.081893,0.069788,0.067553,0.066924,0.067827,0.070162,0.071911,0.072451,0.071782
2,-0.163450,-0.156895,-0.145752,-0.136284,-0.122597,-0.098510,-0.067956,-0.033574,0.005142,0.044869,...,0.087766,0.081893,0.069788,0.067553,0.066924,0.067827,0.070162,0.071911,0.072451,0.071782
3,-0.163462,-0.156908,-0.145755,-0.136281,-0.122597,-0.098517,-0.067972,-0.033580,0.005137,0.044863,...,0.087772,0.081898,0.069807,0.067551,0.066919,0.067826,0.070184,0.071954,0.072516,0.071826
4,-0.163406,-0.156854,-0.145720,-0.136245,-0.122544,-0.098467,-0.067924,-0.033558,0.005157,0.044874,...,0.087870,0.081942,0.069783,0.067510,0.066874,0.067786,0.070165,0.071976,0.072554,0.071843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,-0.148756,-0.148653,-0.147350,-0.142510,-0.125884,-0.099116,-0.067693,-0.034620,0.002989,0.042856,...,0.087104,0.081242,0.062722,0.061722,0.062258,0.058505,0.057794,0.058807,0.063288,0.062188
961,-0.148750,-0.148671,-0.147348,-0.142510,-0.125914,-0.099120,-0.067702,-0.034628,0.002981,0.042855,...,0.087195,0.081358,0.062807,0.061829,0.062320,0.058664,0.057920,0.058891,0.063298,0.062257
962,-0.148761,-0.148685,-0.147358,-0.142514,-0.125920,-0.099120,-0.067688,-0.034631,0.002974,0.042849,...,0.087149,0.081329,0.062785,0.061839,0.062338,0.058680,0.057926,0.058856,0.063253,0.062230
963,-0.148763,-0.148652,-0.147356,-0.142512,-0.125875,-0.099113,-0.067682,-0.034619,0.002988,0.042853,...,0.087044,0.081181,0.062667,0.061684,0.062242,0.058448,0.057757,0.058756,0.063262,0.062147


In [3]:
cant_frames_per_seq = pd.read_csv('ckmock_landmark_FramePerSeq.csv',header = None)
cant_frames_per_seq =np.asarray(cant_frames_per_seq).flatten()

In [4]:
ind =0
my_data_aligned_diff =[]
for i in range(0, len(cant_frames_per_seq)):#51
    cant_frames = cant_frames_per_seq[i]
    frame_zero = my_data.iloc[ind]    
    for j in range(ind, ind + cant_frames):
        my_data_aligned_diff.append(my_data.iloc[j]- my_data.iloc[ind]) #no dice nada de abs
    ind = ind + cant_frames
    
my_data_aligned_diff = np.asarray(my_data_aligned_diff)

In [7]:
my_df = pd.DataFrame(my_data_aligned_diff)
my_df.to_csv('my_data_aligned_diff.csv',header = False, index= False)

# Ahora hacemos PCA

En el paper nos dice que restaramos el mean por columna, nosotros tambien normalizamos por standard deviation (dividimos).
La cantidad de xomponentes es la minima que alcanze un 99% loadings, en este caso 38.

In [17]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
scaled_landDiff = std_scaler.fit_transform(my_data_aligned_diff)
scaled_landDiff.shape

(965, 136)

In [18]:
from sklearn.decomposition import PCA
pca = PCA(n_components=38)
pca.fit_transform(scaled_landDiff)
b_shape = pca.fit_transform(scaled_landDiff)
b_shape.shape # cada fila deberia ser el b de 1 frame en 38 componentes proncipales (de 136 2*l landmarks)

(965, 38)

In [19]:
print(sum(pca.explained_variance_ratio_))


0.9907696428747979


 cada fila deberia ser el b de 1 frame en 38 componentes proncipales (de 136 2*l landmarks)

## Aca deberiamos graficar el impacto de los principal componentes.